In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta  ## use timedelta to increment targ by 1 year!!!!
#from datetime import timedelta  ## use this to increment targ by 1 year!!!!
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices
#import random

import sys
import os
os.chdir('C:/BankRisk')

In [2]:
dur20y = pd.DataFrame(columns=['dur','adj'])  ## used at end to save last 20y act vs naive durations of ladders
dur20y

Empty DataFrame
Columns: [dur, adj]
Index: []

In [3]:
sp30y = pd.read_csv("ccs_all365_longhist_30yr.csv",index_col=0)
sp30y.index = pd.to_datetime(sp30y.index)
sp30y.columns = sp30y.columns.astype(int)

In [4]:
sp30y.tail(2)

0         1         2         3         4         5    \
Date                                                                     
2016-11-01  0.415694  0.570931  0.751203  0.921077  1.079826  1.224034   
2016-12-01  0.547500  0.721848  0.862734  0.988876  1.129987  1.264519   

                 6         7         8         9      ...          351  \
Date                                                  ...                
2016-11-01  1.286939  1.268851  1.223631  1.164845    ...     2.289467   
2016-12-01  1.324473  1.312911  1.284005  1.246427    ...     2.628734   

                 352       353       354       355       356       357  \
Date                                                                     
2016-11-01  2.290383  2.291298  2.292213  2.293128  2.294043  2.294957   
2016-12-01  2.629494  2.630255  2.631015  2.631775  2.632534  2.633294   

                 358       359       360  
Date                                      
2016-11-01  2.295871  2.296786  2.297700  
2016-12-01  2.634053  2.634812  2.635571  

[2 rows x 361 columns]

In [5]:
#df_sp = pd.read_csv("df_spot_longhist_30y.csv",index_col=0)
df_sp = pd.read_csv("df_spot_fwd3_longhist_30y.csv",index_col=0)
df_sp['Date'] = pd.to_datetime(df_sp['Date'])
df_sp.reset_index(drop=False,inplace=True)
df_sp.set_index(['Desc','Date'],drop=True,inplace=True)
df_sp.columns = df_sp.columns.astype(int)
df_sp.tail(2)

0         1         2         3         4         5    \
Desc Date                                                                     
fwd3 2016-11-01  0.920019  1.246917  1.534383  1.645614  1.512567  1.214907   
     2016-12-01  0.987656  1.263276  1.527256  1.652558  1.547968  1.307443   

                      6         7         8         9   ...        351  \
Desc Date                                               ...              
fwd3 2016-11-01  0.913725  0.720224  0.621170  0.678277 ...   2.749506   
     2016-12-01  1.081698  0.957549  0.893787  0.936202 ...   3.025426   

                      352       353       354       355       356       357  \
Desc Date                                                                     
fwd3 2016-11-01  2.752189  2.755224  2.757571  2.760448  2.763024  2.766345   
     2016-12-01  3.027965  3.030608  3.032023  3.034349  3.036619  3.040176   

                 358  359  360  
Desc Date                       
fwd3 2016-11-01  NaN  NaN  NaN  
     2016-12-01  NaN  NaN  NaN  

[2 rows x 361 columns]

In [6]:
############################################################
############################################################
## ladder dur, MV section:

for x in [1,2,3,4,5,6,7,8,9,10,12,15,20,25,30]:
  mt = 12*x  ## swap maturity: use 1,2,3,4,5,6,7,8,9,10,12,15,20 (all * 12 to convert to months)

  ci = [x for x in np.arange(1,mt+1) if x%6==(mt%6)]  # get coupon dates for new swap
  #ci[-5:]
  t2 = pd.Series(np.zeros(mt),index=np.arange(1,mt+1))  # temp table for processing
  #t2[-5:]
  ## start sections to calc ladder durations and MVs:
  #ci[-5:]
  for i in np.arange(mt): # fill up cash flows in 1st possible ladder
    cp = sp30y.ix[i,mt]/2 # get coupon
    for j in np.arange(1,mt): # move prior month cash flows up by 1 month
      t2.loc[j] = t2.loc[j+1]
    t2.loc[mt] = 0        # zero out last cash flow
    t2[ci] = t2[ci]+cp    # assign coupons to payment periods
    t2[mt] = t2[mt]+100   # add return of principal to maturity period
  # tm table for holding cfs:
  tm = pd.DataFrame(t2,columns=[sp30y.index[mt-1].strftime('%Y-%m-%d')]).T
  tm.index = pd.to_datetime(tm.index)
  #tm
  for i in np.arange(mt,len(sp30y)): # process another month:
    for j in np.arange(1,mt): # move prior month cash flows up by 1 month
      t2.loc[j] = t2.loc[j+1]
    t2.loc[mt] = 0        # zero out last cash flow
    cp = sp30y.ix[i,mt]/2 # get coupon for new swap
    t2[ci] = t2[ci]+cp    # add new coupons to payment periods
    t2[mt] = t2[mt]+100   # add return of principal to maturity period
    d2 = pd.DataFrame(t2,columns=[sp30y.index[i].strftime('%Y-%m-%d')]).T
    d2.index = pd.to_datetime(d2.index)
    tm=tm.append(d2)
  tm.to_csv(str(mt)+'mLadder_cfs_longhist2.csv')
  #str(mt)+'mLadder_cfs_longhist2.csv'

  # calc MacDurs and MVs:
  ts = list(tm.columns)
  ts = [x/12 for x in ts]
  #ts[-5:]  # list of yrs for dur calcs
  dv = pd.DataFrame(tm[[1,2,3,4]])
  dv.columns = ['mv','mvpc','dur','cpn']
  dv[['mv','mvpc','dur','cpn']] = [0,0,0,0]
  #dv.head()
  ## calc dur, mv, mv% for all ladders:
  for i in np.arange(len(tm)):
    pv= [x*y for x,y in zip(tm.ix[i][0:mt],list(df_sp.loc['df',tm.index[i]])[1:mt+1])] ## mult 2 vectors: cf*df
    pvt= [x*y for x,y in zip(pv,ts)]
    dv.ix[i]['dur'] = sum(pvt)/sum(pv)
    dv.ix[i]['mv'] = sum(pv) - 100*mt
    dv.ix[i]['mvpc'] = sum(pv) / (100*mt)
    dv.ix[i]['cpn'] = sp30y.ix[dv.index[i]][mt] # pull coupon for swap maturity mt
  ## add fed funds column to first section for used in case study later:
  if (mt == 12)&('ff' not in list(dv.columns)):
    dvcol = ['mv','mvpc','dur','cpn']
    dv = dv.merge(sp30y[[0]],how='left',left_index=True,right_index=True)
    dv.rename(columns={0:'ff'},inplace=True)
    dv = dv[['ff']+dvcol]
  
  #dv.tail(3)
  ## calc MV3m:  
  b3 = mt/12  # balance per month repricing
  mv3 = pd.DataFrame(sp30y[3])
  mv3.rename(columns={3:'rt'},inplace=True)
  df3 = mv3.merge(df_sp.loc['df'][[1,2,3]],how='left',left_index=True,right_index=True)
  df3['cf1'] = df3['rt'].shift(2) * b3 + mt*100/3
  df3['cf2'] = df3['rt'].shift(1) * b3 + mt*100/3
  df3['cf3'] = df3['rt'] * b3 + mt*100/3
  df3['mv3'] = df3[1]*df3['cf1']+df3[2]*df3['cf2']+df3[3]*df3['cf3'] - mt*100
  #df3.head()
  dv['mv3'] = df3['mv3']
  #dv.head()
  roll3m = pd.DataFrame(pd.rolling_mean(sp30y[3],3, min_periods=3))
  roll3m.columns = ['roll3m']
  sp30ya = pd.rolling_mean(sp30y[mt],mt, min_periods=mt)
  sp30ya.dropna(inplace=True)
  #sp30ya.head()
  #roll3m = pd.DataFrame(pd.rolling_mean(sp30y[3],3, min_periods=3))
  #roll3m.columns = ['roll3m']
  #roll3m.head()
  dv['roll'+str(mt)+'m'] = sp30ya
  dv = dv.merge(roll3m, how='left',left_index=True, right_index=True)
  #dv.tail()
  dv.to_csv('durmv_'+str(mt)+'mladder_longhist2.csv')
  #'durmv_'+str(mt)+'mladder_longhist2.csv'
  ### save last calc 20y dur vs naive dur in a table
  dur20y.loc[mt] = [ dv['dur'][-240:].mean(),  dv['dur'][-240:].mean()/(mt/12/2) ]
  dur20y.sort_index(inplace=True)
  #dur20y
  ## save copies of results according to orig maturity:
  if mt == 12:
    dv12 = dv.copy()
  elif mt == 24:
    dv24 = dv.copy()
  elif mt == 36:
    dv36 = dv.copy()
  elif mt == 48:
    dv48 = dv.copy()
  elif mt == 60:
    dv60 = dv.copy()
  elif mt == 72:
    dv72 = dv.copy()
  elif mt == 84:
    dv84 = dv.copy()
  elif mt == 96:
    dv96 = dv.copy()
  elif mt == 108:
    dv108 = dv.copy()
  elif mt == 120:
    dv120 = dv.copy()
  elif mt == 144:
    dv144 = dv.copy()
  elif mt == 180:
    dv180 = dv.copy()
  elif mt == 240:
    dv240 = dv.copy()
  elif mt == 300:
    dv300 = dv.copy()
  elif mt == 360:
    dv360 = dv.copy()


In [7]:
### STOP HERE!!! run all maturities before proceeding

In [8]:
dur20y.to_csv('durladder_last20y_adjmnts.csv')

In [9]:
#voldet.columns = pd.MultiIndex.from_product([[prn], list(voldet.columns)])
dv12.columns = pd.MultiIndex.from_product([['dv12'],list(dv12.columns)])
dv24.columns = pd.MultiIndex.from_product([['dv24'],list(dv24.columns)])
dv36.columns = pd.MultiIndex.from_product([['dv36'],list(dv36.columns)])
dv48.columns = pd.MultiIndex.from_product([['dv48'],list(dv48.columns)])
dv60.columns = pd.MultiIndex.from_product([['dv60'],list(dv60.columns)])
dv72.columns = pd.MultiIndex.from_product([['dv72'],list(dv72.columns)])
dv84.columns = pd.MultiIndex.from_product([['dv84'],list(dv84.columns)])
dv96.columns = pd.MultiIndex.from_product([['dv96'],list(dv96.columns)])
dv108.columns = pd.MultiIndex.from_product([['dv108'],list(dv108.columns)])
dv120.columns = pd.MultiIndex.from_product([['dv120'],list(dv120.columns)])
dv144.columns = pd.MultiIndex.from_product([['dv144'],list(dv144.columns)])
dv180.columns = pd.MultiIndex.from_product([['dv180'],list(dv180.columns)])
dv240.columns = pd.MultiIndex.from_product([['dv240'],list(dv240.columns)])
dv300.columns = pd.MultiIndex.from_product([['dv300'],list(dv300.columns)])
dv360.columns = pd.MultiIndex.from_product([['dv360'],list(dv360.columns)])

In [10]:
dv12.head()

dv12                                                     \
                  ff         mv      mvpc       dur       cpn       mv3   
1965-12-01  4.380000   9.872790  1.008227  0.534467  5.493205  4.784516   
1966-01-01  4.481389   9.288667  1.007741  0.534508  5.463205  4.955346   
1966-02-01  4.663889   9.148350  1.007624  0.534340  5.713205  5.366464   
1966-03-01  4.714583  10.391184  1.008659  0.534491  5.503205  5.672123   
1966-04-01  4.734861  11.174960  1.009312  0.534420  5.533205  5.637323   

                                
             roll12m    roll3m  
1965-12-01  4.854038  4.998342  
1966-01-01  4.916538  5.187601  
1966-02-01  4.990705  5.376861  
1966-03-01  5.053205  5.447833  
1966-04-01  5.117372  5.457972

In [11]:
dvtot = dv12.copy()
dvtot = dvtot.merge(dv24,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv36,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv48,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv60,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv72,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv84,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv96,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv108,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv120,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv144,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv180,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv240,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv300,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv360,how='left',left_index=True,right_index=True)

In [12]:
dvtot.tail()

dv12                                                    \
                  ff        mv      mvpc       dur       cpn       mv3   
2016-08-01  0.405556  0.205113  1.000171  0.540488  0.910000  0.747136   
2016-09-01  0.405556  0.214677  1.000179  0.540445  0.950000  0.885025   
2016-10-01  0.405556  0.404636  1.000337  0.540378  0.987650  1.027131   
2016-11-01  0.415694  0.501120  1.000418  0.540326  1.042750  1.060425   
2016-12-01  0.547500  0.349288  1.000291  0.540241  1.168381  0.995301   

                                    dv24              ...         dv300  \
             roll12m    roll3m        mv      mvpc    ...           mv3   
2016-08-01  0.730833  0.729750  2.626708  1.001094    ...     18.678411   
2016-09-01  0.765833  0.796680  1.932479  1.000805    ...     22.125626   
2016-10-01  0.807304  0.858314  1.508241  1.000628    ...     25.678284   
2016-11-01  0.840867  0.891141  0.207704  1.000087    ...     26.510635   
2016-12-01  0.873232  0.933613 -2.588365  0.998922    ...     24.882536   

                                       dv360                                 \
            roll300m    roll3m            mv      mvpc        dur       cpn   
2016-08-01  5.307572  0.729750  16159.478329  1.448874  11.183230  1.750000   
2016-09-01  5.285832  0.796680  15761.708418  1.437825  11.151701  1.800000   
2016-10-01  5.264230  0.858314  14882.603055  1.413406  11.051317  1.946950   
2016-11-01  5.244017  0.891141  12816.749838  1.356021  10.809251  2.297700   
2016-12-01  5.226575  0.933613  10900.371179  1.302788  10.586855  2.635571   

                                           
                  mv3  roll360m    roll3m  
2016-08-01  22.414093  6.033302  0.729750  
2016-09-01  26.550751  6.014969  0.796680  
2016-10-01  30.813941  5.996826  0.858314  
2016-11-01  31.812762  5.980163  0.891141  
2016-12-01  29.859044  5.964862  0.933613  

[5 rows x 106 columns]

In [13]:
dvtot.to_csv('ladder_durmv_allmat_longhist.csv')

In [14]:
## methods to access multiindex data:
dvtot['dv24','mv'][10:15]  # one column

1966-10-01          NaN
1966-11-01          NaN
1966-12-01    27.638363
1967-01-01    35.049880
1967-02-01    34.315623
Name: (dv24, mv), dtype: float64

In [15]:
## methods to access multiindex cell:
dvtot.ix['2015-09-01']['dv24','dur']  # one cell  (loc works too)

1.0371136056624237

In [16]:
dur20y

dur       adj
12   0.537588  1.075175
24   1.021879  1.021879
36   1.492252  0.994835
48   1.947063  0.973531
60   2.385462  0.954185
72   2.806983  0.935661
84   3.211180  0.917480
96   3.597944  0.899486
108  3.966878  0.881528
120  4.321228  0.864246
144  4.993103  0.832184
180  5.902958  0.787061
240  7.173260  0.717326
300  8.212457  0.656997
360  9.098412  0.606561